In [398]:
# importo le librerie
import pandas as pd
import numpy as np
import pulp

# importo le funzioni
from os.path import join
from itertools import product

In [399]:
# scelta del track
choices = ['MCS','MMF','MST']
track = choices[2]
print(f'You chose {track}')

You chose MST


In [438]:
# DATAFRAME CORSI OBBLIGATORI
PATH1 = join('assets','source_'+track+'_1.xlsx')
PATH2 = join('assets','source_'+track+'_2.xlsx')
# comp = compulsory
df_comp1 = pd.read_excel(PATH1).dropna()
df_comp2 = pd.read_excel(PATH2).dropna()
comp1 = len(df_comp1)
comp2 = len(df_comp2)
df_comp = pd.concat([df_comp1, df_comp2]) # unisco i compulsory dei due anni
df_comp.reset_index(inplace=True, drop=True) # reset dell'index 0-4,0-1

df_comp.head()

,Denominazione,Codice,Sem,CFU,Gruppo,Rating
0,REAL AND FUNCTIONAL ANALYSIS,95958,1,8,MST,0
1,STOCHASTIC DYNAMICAL MODELS,54074,1,8,MST,0
2,ALGORITHMS AND PARALLEL COMPUTING,52496,1,10,MST,0
3,MODEL IDENTIFICATION AND DATA ANALYSIS,96297,2,10,MST,0
4,APPLIED STATISTICS,52498,2,10,MST,0


In [453]:
#GROUPS_names = ['CSCL','FREE','ING','MTM','STAT'] # gruppi
#GROUPS_names.append(track) # aggiungo il gruppo fittizio del track
#
#CFU_costraints = { # CSCL FREE ING MTM STAT TRACK
#    'MCS':           [10, 16,  10, 16,  0,  0],
#    'MMF':           [ 0, 16,  10, 16,  0,  0],
#    'MST':           [ 0, 16,  10, 16, 10,  0]
#}
#
#min_cfus = CFU_costraints[track]
#print(f'Minimum number of CFUs for {track}:\n{[(i,j) for (i,j) in zip(GROUPS_names,min_cfus)]}')
#print(GROUPS_names)

In [452]:
PATH_CFU_COSTRAINTS = join('assets','source_min_CFUs.xlsx')
df_cfu = pd.read_excel(PATH_CFU_COSTRAINTS,index_col=0).dropna()
df_cfu[track] = 0 # aggiungo colonna il gruppo del track con minimo 0 CFU

GROUPS_names = df_cfu.columns.tolist()
min_cfus = df_cfu.loc[track].tolist()

['CSCL', 'FREE', 'ING', 'MTM', 'STAT', 'MST']

In [402]:
# importo il dataframe utente
dfu = pd.read_excel("Source_Polimi.xlsx").dropna()


df = dfu #pd.concat([df_comp, dfu])



In [404]:
# creo un dataframe per dare corrispondenza codice-denominazione
code_name = pd.concat([df_comp, dfu]).set_index('Codice')
code_name.drop(columns=['Sem','CFU','Gruppo','Rating'],inplace=True)
#code_name.loc[56955]['Denominazione']
code_name = code_name[~code_name.index.duplicated(keep='last')] # specificare quale tenere perché potrebbero a priori avere "Denominazione" diversa, ma non in questo caso perché uguale codice => uguale nome
code_name = code_name.to_dict()['Denominazione']

#code_name

In [405]:
df_comp = df_comp.drop(columns=['Denominazione'])

In [406]:
# creo un df in cui sparisce la denominazione e tengo codice, sem, cfu, rating, e i gruppi vanno in una lista



In [407]:
# in questo modo scompaiono anche tutte le righe senza rating (giusto: non le vogliamo)
df2 = pd.DataFrame(df.groupby(['Codice','Sem','CFU','Rating'])['Gruppo'].apply(list))

In [408]:
df_comp

,Codice,Sem,CFU,Gruppo,Rating
0,95958,1,8,MST,0
1,54074,1,8,MST,0
2,52496,1,10,MST,0
3,96297,2,10,MST,0
4,52498,2,10,MST,0
5,52499,1,10,MST,0
6,97690,2,12,MST,0


In [409]:
df2.reset_index(inplace=True)

In [410]:
#warning
df2 = pd.concat([df_comp, df2])

In [411]:
df2.reset_index(inplace=True)
df2.drop(columns=['index'],inplace=True)
#df2.drop(columns=['level_0'],inplace=True)

In [412]:
# TODO bisognerebbe creare un df da esportare con già i raggruppamenti

In [413]:
dfexplode = df2.explode('Gruppo')
dfexplode.head()

,Codice,Sem,CFU,Gruppo,Rating
0,95958,1,8,MST,0.0
1,54074,1,8,MST,0.0
2,52496,1,10,MST,0.0
3,96297,2,10,MST,0.0
4,52498,2,10,MST,0.0


In [414]:
#dfcrosstab = pd.crosstab(dfexplode['Codice'], dfexplode['Gruppo']).ne(0).reset_index()
dfcrosstab = pd.crosstab(dfexplode['Codice'], dfexplode['Gruppo'])
#dfcrosstab.loc[52503]['FREE']
dfcrosstab.head()
#dfcrosstab.dtypes

Gruppo,CSCL,FREE,ING,MST,MTM,STAT
Codice,,,,,,
51132,0,1,0,0,0,1
51197,0,1,1,0,0,0
52354,0,1,1,0,0,0
52397,0,1,0,0,0,0
52496,0,0,0,1,0,0


In [415]:
dict_base = df2.to_dict()

In [416]:
# PARAMETRI
codes    = dict_base['Codice']
semester = dict_base['Sem']
cfus     = dict_base['CFU']
rating   = dict_base['Rating']

In [417]:
# INSIEMI

COURSES = list(codes.keys())
YEARS   = [1, 2]
GROUPS  = list(range(len(GROUPS_names)))

GROUPS

[0, 1, 2, 3, 4, 5]

In [418]:
# PROBLEMA
prob = pulp.LpProblem('Piano_di_studi', pulp.const.LpMaximize)

In [419]:
# VARIABILI
x = pulp.LpVariable.dicts('corso-anno-gruppo', (COURSES, YEARS, GROUPS), lowBound=0, upBound=1, cat='Continuous')  # quanto corso i anno j gruppo k
y = pulp.LpVariable.dicts('corso',             (COURSES),                lowBound=0, upBound=1, cat='Binary')      # 1 se corso i è scelto
z = pulp.LpVariable.dicts('corso-anno',        (COURSES, YEARS),         lowBound=0, upBound=1, cat='Binary')      # 1 se corso i all'anno j

In [420]:
# FUNZIONE OBIETTIVO
prob += pulp.lpSum([rating[i]*cfus[i]*x[i][j][k] for (i, j, k) in product(COURSES, YEARS, GROUPS)])

In [421]:
CFU_max_sem=35
CFU_max_tot=121 # il max di quelli a scelta (obbligatori sono 68)

In [422]:
# VINCOLI

# obbligatorietà dei corsi
for i in range(comp1):
    # 5 dovrebbe essere l'indice dell'ultimo gruppo, quello "finto" della track
    prob += x[i][1][GROUPS[-1]] == 1, f"Compulsory course {i} in year 1"

for i in range(comp1, comp1+comp2):
    prob += x[i][2][GROUPS[-1]] == 1, f"Compulsory course {i} in year 2"

# max cfu totali
prob += pulp.lpSum([cfus[i]*x[i][j][k] for (i, j, k) in product(COURSES, YEARS, GROUPS)]) <= CFU_max_tot, f"Maximum total CFUs"
#prob += pulp.lpSum([cfus[i]*y[i] for i in COURSES]) <= CFU_max_tot, f"Maximum total CFUs"

# max cfu per semestre per anno
for j in YEARS:
    group_courses=[i for i in COURSES if semester[i] == 1]
    prob += pulp.lpSum([cfus[i]*x[i][j][k] for (i,k) in product(group_courses,GROUPS)]) <= CFU_max_sem, f"Maximum CFUs for year {j} semester 1"

    group_courses=[i for i in COURSES if semester[i] == 2]
    prob += pulp.lpSum([cfus[i]*x[i][j][k] for (i,k) in product(group_courses,GROUPS)]) <= CFU_max_sem, f"Maximum CFUs for year {j} semester 2"


# (logico) se un corso è scelto allora z_ij = 1
for (i,j) in product(COURSES,YEARS):
    prob += pulp.lpSum([x[i][j][k] for k in GROUPS]) <= 1000 * z[i][j], f"If course {i} chosen, then z[{i}][{j}]=1"

# (capacità) non posso scegliere un corso in più anni (cioè mettere GAT metà al primo e metà al secondo annno per es.)
for i in COURSES:
    prob += pulp.lpSum([z[i][j] for j in YEARS]) <= 1, f"Course {i} in no more than one year"

# (logico) se un corso è scelto allora y[i]=1
for i in COURSES:
    prob += pulp.lpSum([x[i][j][k] for (j, k) in product(YEARS, GROUPS)]) <= y[i], f"If course {i} chosen, then y[{i}]=1"

# TODO SUS

# (assegnamento) se y_i=1 allora tutti quanti sopra tutti i gruppi devono sommare a 1 (circa)
for i in COURSES:
    prob += y[i] <= pulp.lpSum([x[i][j][k] for (j, k) in product(YEARS, GROUPS)]), f"If course {i} chosen, then sum of x[{i}][j][k]=1"

# (capacità) corso scelto al massimo una sola volta
#for i in COURSES:
#    prob += pulp.lpSum([x[i][j][k] for (j, k) in product(YEARS, GROUPS)]) <= 1, f"Course {i} chosen at most 1 time"


In [423]:
# min cfu gruppo (nessun vincolo per l'ultimo che è quello della track)
for k in GROUPS[:-1]:
    prob += pulp.lpSum([cfus[i]*x[i][j][k] for (i,j) in product(COURSES,YEARS)]) >= min_cfus[k], f"Minimum CFUs for group {k}"  # >=/==


# dfcrosstab.loc[52503]['FREE']
# dfcrosstab.loc[codes[...]][GROUPS_names[...]]

# (capacità) posso dare peso a un corso in un gruppo solo è consentito
for (i,k) in product(COURSES,GROUPS):
    prob += pulp.lpSum([x[i][j][k] for j in YEARS]) <= dfcrosstab.loc[codes[i]][GROUPS_names[k]], f"Possibility to choose course {i} in group {k} only if allowed"

---

In [424]:
# RISOLUZIONE
status = prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/teo/Documents/GitHub/compilatore-piano-di-studi/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/1n/5l6269bd647d957r5df7_hj80000gn/T/f65b538bab4c4c57a9f2b2b7e798fc44-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/1n/5l6269bd647d957r5df7_hj80000gn/T/f65b538bab4c4c57a9f2b2b7e798fc44-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 660 COLUMNS
At line 6732 RHS
At line 7388 BOUNDS
At line 8259 ENDATA
Problem MODEL has 655 rows, 870 columns and 5111 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 253 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 59 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0004I processed model has 323 rows, 321 columns (153 integ

In [426]:
print(f"Status:    {pulp.LpStatus[status]}")
print(f"Objective: {prob.objective.value()}")

Status:    Optimal
Objective: 249.0


In [ ]:
prob.constraints

In [ ]:
prob.writeLP('filename.txt')

In [432]:
# output
# ne servono 2
# - SOMMARIO, quello che uno ha bisogno di sapere, con le variabili z, quale corso in quale anno
# - DETTAGLI, quello da dare a Gregoratti, quale percentuale di quale corso in quale gruppo, con le x

corsi=[]
for (i,j,k) in product(COURSES, YEARS, GROUPS):
    val = x[i][j][k].varValue
    if val and val > 0:
        #codice = codes[i]
        #gruppo = GROUPS_names[k]
        #anno = YEARS[j-1]
        #string = f'{val*100:3.0f}% del corso: {codice}\t\tnel gruppo {gruppo}\t\tnell\'anno {anno}'
        #print(string)
                           
        course_name=code_name[codes[i]]
        semester_name=semester[i]
        cfus_name=cfus[i]
        codes_name=codes[i]
        group_name=GROUPS_names[k]
        year_name = YEARS[j-1]
        percentage = val
        el=[codes_name, course_name, cfus_name, group_name, year_name, semester_name, val]
        corsi.append(el)
        print(f"{val*100:8.3f}% di {course_name} [{group_name}, {cfus_name} CFU] all'anno {year_name} al semestre {semester_name}")
        # :3.0f

piano=pd.DataFrame(corsi, columns=['Codice', 'Denominazione', 'CFU', 'Gruppo', 'Anno', 'Sem','%'])
piano=piano.set_index('Codice')
piano=piano.sort_values(['Anno', 'Sem'], ascending=[True, True])

piano#[piano['Gruppo']=='FREE']

 100.000% di REAL AND FUNCTIONAL ANALYSIS [MST, 8 CFU] all'anno 1 al semestre 1
 100.000% di STOCHASTIC DYNAMICAL MODELS [MST, 8 CFU] all'anno 1 al semestre 1
 100.000% di ALGORITHMS AND PARALLEL COMPUTING [MST, 10 CFU] all'anno 1 al semestre 1
 100.000% di MODEL IDENTIFICATION AND DATA ANALYSIS [MST, 10 CFU] all'anno 1 al semestre 2
 100.000% di APPLIED STATISTICS [MST, 10 CFU] all'anno 1 al semestre 2
 100.000% di BAYESIAN STATISTICS [MST, 10 CFU] all'anno 2 al semestre 1
 100.000% di FINAL WORK [MST, 12 CFU] all'anno 2 al semestre 2
  25.000% di GAME THEORY [FREE, 8 CFU] all'anno 2 al semestre 1
  75.000% di GAME THEORY [MTM, 8 CFU] all'anno 2 al semestre 1
 100.000% di ADVANCED PROGRAMMING FOR SCIENTIFIC COMPUTING [MTM, 10 CFU] all'anno 2 al semestre 2
 100.000% di ARTIFICIAL NEURAL NETWORKS AND DEEP LEARNING [STAT, 5 CFU] all'anno 2 al semestre 1
 100.000% di ADVANCED PROGRAMMING FOR SCIENTIFIC COMPUTING [FREE, 8 CFU] all'anno 2 al semestre 2
 100.000% di COMPUTATIONAL STATISTICS 

,Denominazione,CFU,Gruppo,Anno,Sem,%
Codice,,,,,,
95958,REAL AND FUNCTIONAL ANALYSIS,8,MST,1,1,1.00
54074,STOCHASTIC DYNAMICAL MODELS,8,MST,1,1,1.00
52496,ALGORITHMS AND PARALLEL COMPUTING,10,MST,1,1,1.00
55701,COMPUTATIONAL STATISTICS,5,STAT,1,1,1.00
96297,MODEL IDENTIFICATION AND DATA ANALYSIS,10,MST,1,2,1.00
52498,APPLIED STATISTICS,10,MST,1,2,1.00
56894,ONLINE LEARNING APPLICATIONS,5,ING,1,2,1.00
56981,ALGORITHMIC GAME THEORY,6,FREE,1,2,1.00
52499,BAYESIAN STATISTICS,10,MST,2,1,1.00
